In [1]:
include("../../../julia/FNC_init.jl")

  Activating 

project at `~/Documents/GitHub/fnc`


[**Demo %s**](#demo-pivoting-fail)

Here is a previously encountered matrix that factors well.

In [2]:
A = [2 0 4 3 ; -4 5 -7 -10 ; 1 15 2 -4.5 ; -2 0 2 -13];
L, U = FNC.lufact(A)
L

4×4 LowerTriangular{Float64, Matrix{Float64}}:
  1.0   ⋅     ⋅    ⋅ 
 -2.0  1.0    ⋅    ⋅ 
  0.5  3.0   1.0   ⋅ 
 -1.0  0.0  -2.0  1.0

If we swap the second and fourth rows of $\mathbf{A}$, the result is still nonsingular. However, the factorization now fails.

In [3]:
A[[2, 4], :] = A[[4, 2], :]  
L, U = FNC.lufact(A)
L

4×4 LowerTriangular{Float64, Matrix{Float64}}:
  1.0     ⋅      ⋅    ⋅ 
 -1.0  NaN       ⋅    ⋅ 
  0.5   Inf   NaN     ⋅ 
 -2.0   Inf   NaN    1.0

```{index} Julia; NaN
```

The presence of `NaN` in the result indicates that some impossible operation was required. The source of the problem is easy to locate. We can find the first outer product in the factorization just fine:

In [4]:
U[1, :] = A[1, :]
L[:, 1] = A[:, 1] / U[1, 1]
A -= L[:, 1] * U[1, :]'

4×4 Matrix{Float64}:
 0.0   0.0  0.0    0.0
 0.0   0.0  6.0  -10.0
 0.0  15.0  0.0   -6.0
 0.0   5.0  1.0   -4.0

The next step is `U[2, :] = A[2, :]`, which is also OK. But then we are supposed to divide by `U[2, 2]`, which is zero. The algorithm cannot continue.